<a href="https://colab.research.google.com/github/danjgmoyo/EE_Notebooks/blob/main/SENTINEL_2_DOWNLOADER_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import modules and setting aup neccessary environments

In [1]:
import os
import ee
import geemap

ee.Authenticate()
ee.Initialize(project = "danjgmaps")
Map = geemap.Map()

Import or drawing region of interest

In [2]:
mzimba_district = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2") \
    .filter(ee.Filter.eq('ADM2_NAME', 'Mzimba'))

# Checking if the import was successful or not
print(mzimba_district.size().getInfo())

1


setting up parametes

In [3]:

start_date = '2024-04-02'
end_date = '2024-12-02'
ROI = mzimba_district.geometry()
MAX_CLOUD_COVER = 10

IMAGE PROCESSING

In [4]:
def mask_clouds_shadows(image):
    """Advanced cloud/shadow masking using QA60 band and scene metadata"""
    # Get QA60 band and create cloud/shadow masks
    qa = image.select('QA60')

    # Bitwise operations for cloud detection (bits 10 and 11)
    cloud_mask = (1 << 10)  # Opaque clouds
    cirrus_mask = (1 << 11)  # Cirrus clouds
    mask = (
        qa.bitwiseAnd(cloud_mask).eq(0)
        .And(qa.bitwiseAnd(cirrus_mask).eq(0))
    )

    # Apply scaling and masks
    return image.updateMask(mask).divide(10000)


def prepare_analysis_ready_collection():
    """Creates cloud-masked analysis-ready image collection"""
    sent_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(ROI)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', MAX_CLOUD_COVER))
        .sort('CLOUDY_PIXEL_PERCENTAGE', True)
        .map(mask_clouds_shadows)
        .select(['B2', 'B3', 'B4', 'B8', 'B11'])  # Visible, NIR, SWIR
    )

    return sent_collection

SOME WEIRD TROUBLESHOOTING AND IMAGE SELECTION EITHER A MEDIAN OR NOT

In [7]:
sent2_collection = prepare_analysis_ready_collection()
# sent_collection.size().getInfo()
median_collection = sent2_collection.median()


first_image = sent2_collection.first()
# first_image.getInfo()


In [8]:
Map.centerObject(ROI, 10)
Map.addLayer(median_collection, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Median')
Map


Map(bottom=140384.0, center=[-11.770983125351538, 33.63769557063868], controls=(WidgetControl(options=['positi…

EXPORTS

In [ ]:
composite = median_collection.clip(ROI).set({
    'system:band_names': ['B2', 'B3', 'B4', 'B8', 'B11', 'QA60'],
    'scale_factor': 10000,
    'description': 'Sentinel-2 Analysis Ready Data'

})

task = ee.batch.Export.image.toDrive(
    image=composite,
    description='Sentinel2_Export',
    folder='GEE_SENTINEL',
    fileNamePrefix='Mzimba',
    region=ROI,
    scale=10,
    crs='EPSG:4326',
    fileFormat='GeoTIFF',
    formatOptions={
        'cloudOptimized': True,
        'noData': 0

    },
    maxPixels=1e13
)

task.start()
print("""Export initiated! Key features:
- Preserved band names in metadata
- Original reflectance scaling (0-10000)
- Cloud-optimized GeoTIFF (COG)
- QA60 band included for custom masking

Access your data:
1. Google Drive folder: {drive_folder}
2. File name: {export_name}.tif
3. Monitor task: https://code.earthengine.google.com/tasks

QGIS Analysis Ready:
- NDVI Formula: (B8 - B4) / (B8 + B4)
- No scaling needed - use raw values directly
- Band names visible in Properties → Metadata""")


Export initiated! Key features:
- Preserved band names in metadata
- Original reflectance scaling (0-10000)
- Cloud-optimized GeoTIFF (COG)
- QA60 band included for custom masking

Access your data:
1. Google Drive folder: {drive_folder}
2. File name: {export_name}.tif
3. Monitor task: https://code.earthengine.google.com/tasks

QGIS Analysis Ready:
- NDVI Formula: (B8 - B4) / (B8 + B4)
- No scaling needed - use raw values directly
- Band names visible in Properties → Metadata
